In [1]:
# imports

import chromadb
from chromadb.utils import embedding_functions

import json
import chromadb
from chromadb.config import Settings
from chromadb import EmbeddingFunction
from chromadb.utils import embedding_functions

In [2]:
## Terminal command to start a local host server
## chroma run --path "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data"

## instantiate chroma client
chroma_internet_client = chromadb.HttpClient(host='16.171.68.145', port=8000, settings=Settings(allow_reset=True))

In [3]:
# Function to create multiple collections
persistence_database_path_windows = "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data"

persistence_database_path_mac = "/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/data/chroma_data"

persistence_database_path_ubuntu = "/home/ubuntu/chromadb"

## for local persistent database
# chroma_internet_client = chromadb.PersistentClient(path=persistence_database_path_mac, settings=Settings(allow_reset=True))


def create_chroma_collections(years):
    
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)
    
    for year in years:
        # Specify the path to your JSON file
        #json_file_path = f"G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/data_json/{year}pubmed.json"
        json_file_path =f"/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/data/data_json/{year}pubmed.json"
        # Open the JSON file and load the data
        with open(json_file_path, 'r') as json_file:
            dataset = json.load(json_file)
            
        collection = chroma_internet_client.create_collection(f"{year}pubmed", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})
        
        collection.add(
            ids=[str(entry['PMID']) for entry in dataset],
            documents= [ entry['PMID'] + "<SEP>" +  entry['Author']+ "<SEP>" + entry['Title'] + "<SEP>" + entry['Abstract'] for entry in dataset],
            metadatas=[
                {'author': entry['Author']} for entry in dataset
            ],
        )



In [ ]:
# chroma_internet_client.reset()

In [4]:
years = ['2013']#, '2014', '2015', '2016-2017', '2018', '2019', '2020-1', '2020-2']  

create_chroma_collections(years=years)

/Users/vasu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## access 2013 collection
collection_2013 = chroma_internet_client.get_collection('2013pubmed')

#### Code for getting top 5 articles based on the similarity index

In [ ]:
# from sentence_transformers import SentenceTransformer, util

# sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)

# # User's question
# user_question = 'CASK Disorder'

# # Embed the user's question
# user_question_embedding = sentence_transformer_ef([user_question])[0]

# search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)
# print(user_question)
# # Print the search results
# for i, result in enumerate(search_results['ids'][0]):
#     document_id = result
#     metadata = search_results['metadatas'][0][i]  # Access the corresponding metadata
#     similarity_score = search_results['distances'][0][i]  # Access the corresponding similarity score
#     document = search_results['documents'][0][i]  # Access the corresponding document

#     print(f"PMID: {document_id}")
#     # print(f"Title: {metadata['title']}")
#     print(f"Author: {metadata['author']}")
#     print(document)
#     print(f"Similarity Score: {similarity_score}")
#     print("---------------")


#### code for retrieving top article based on the similarity

In [6]:
from sentence_transformers import SentenceTransformer, util

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)

# User's question
user_question = 'CASK Disorder'

# Embed the user's question
user_question_embedding = sentence_transformer_ef([user_question])[0]

search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)

# Find the index of the context with the maximum similarity score
max_similarity_index = search_results['distances'][0].index(max(search_results['distances'][0]))

# Get the context with the maximum similarity score
context_with_max_similarity = search_results['documents'][0][max_similarity_index]

print("User Question:")
print(user_question)
print("\nContext with Maximum Similarity Score:")
print(context_with_max_similarity)
print("Similarity Score:", max(search_results['distances'][0]))

# Now you can feed the user question and the context with maximum similarity score into the Turbo model
def get_context_with_max_similarity(user_question):
    # Initialize SentenceTransformer model
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)

    # Embed the user's question
    user_question_embedding = sentence_transformer_ef([user_question])[0]

    # Perform the query using Chroma
    search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)

    # Find the index of the context with the maximum similarity score
    max_similarity_index = search_results['distances'][0].index(max(search_results['distances'][0]))

    # Get the context with the maximum similarity score
    context_with_max_similarity = search_results['documents'][0][max_similarity_index]

    return context_with_max_similarity

User Question:
CASK Disorder

Context with Maximum Similarity Score:
24591081<SEP>Avinash M Veerappa<SEP>Genome-wide copy number scan identifies disruption of PCDH11X in developmental dyslexia<SEP>Developmental dyslexia (DD) is a complex heritable disorder with unexpected difficulty in learning to read and spell despite adequate intelligence, education, environment, and normal senses. We performed a whole genome copy number variations (CNV) scan on 11 dyslexic families consisting of 14 dyslexic subjects and 24 non dyslexic members using 1.8 million combined SNP and CNV markers. We found CNVs affecting protocadherin genes in six dyslexics from three families, while none among the non-dyslexic control members showed any CNV in protocadherins. We identified duplications in five cases and a deletion in one case in Xq21.3 region bearing PCDH11X. Unequal recombination between the X-transposed region (XTR) of Yp11.2 and the X chromosome might be causing these structural changes. PCDH11X, expr

In [7]:
chroma_internet_client.list_collections()

[Collection(name=2013pubmed)]